# RMixtComp functional example

## Data

Load the CSV data file as dataframe.

In [ ]:
data <- read.table("mixtcomp-functional-example.csv", sep = ";", header = TRUE)
print(data$Function[1])

## Visualize the data

In [ ]:
library(RMixtComp)

In [ ]:
value <- sapply(data$Function, function(x) convertFunctionalToVector(x)$value)
time <- sapply(data$Function, function(x) convertFunctionalToVector(x)$time)
matplot(time, value, type = "l")

## Clustering with RMixtComp

Define the distribution used for each variable and the associated hyperparameters. The functional model requires 2 hyêrparameters: `nSub` is the number of subregressions into which the function will be decomposed, `nCoeff` is the number of polynomial coefficients of each subregression (2 = line). 

In [ ]:
nSub <- 2
nCoeff <- 2
model <- list(Function = list(type = "Func_CS", 
                              paramStr = paste0("nSub: ", nSub, ", nCoeff: ", nCoeff)))

Define the SEM algorithm's parameters

In [ ]:
algo <- list(nbBurnInIter = 50,
             nbIter = 100,
             nbGibbsBurnInIter = 50,
             nbGibbsIter = 100,
             nInitPerClass = floor(nrow(data)/2),
             nSemTry = 5,
             confidenceLevel = 0.95,
             ratioStableCriterion = 0.99,
             nStableCriterion = 10)

Choose the desired number of classes and the number of runs for each given number of classes.

In [ ]:
nClass <- 1:6
nRun <- 3

In [ ]:
res <- mixtCompLearn(data, model, algo, nClass = 1:6, criterion = "ICL", nRun = 3, nCore = 1,  hierarchicalMode = "no")

## Output's Analysis

### Criterion

This chart represents the criterion value (choose between BIC and ICL) for each model that was built. The higher the value (close to 0) the better the model. 

In [ ]:
plotCrit(res, pkg = "plotly")

Choose the number of classes to study in the following.

In [ ]:
K <- 3
resK <- extractMixtCompObject(res, K)

### Variables

This chart represents the discriminating level of each variable. A high value (close to one) means that the variable is highly discriminating. A low value (close to zero) means that the variable is poorly discriminating.

In [ ]:
plotDiscrimVar(resK, pkg = "plotly")

This graph displays the similarity between every pair of variable. A high value (close to one) means that the two variables provide the same information for the clustering task (i.e. similar partitions). A low value (close to zero) means that the two variables provide some different information for the clustering task (i.e. different partitions).

In [ ]:
heatmapVar(resK)

Now select the variable to study in the following.

In [ ]:
variable <- "Function"

This chart summarizes the distribution of the selected variable. 

In [ ]:
plotDataBoxplot(resK, variable, grl = TRUE, pkg = "plotly")

### Classes

This chart shows the proportion of individuals in each class.

In [ ]:
plotProportion(resK, pkg = "plotly")

This chart represents the similarity level between each pair of classes. A high value (close to one) means that the 2 classes are strongly different (i.e. low overlapping). A low value (close to zero) means that the 2 classes are similar for the clustering task (i.e. high overlapping).

In [ ]:
heatmapClass(resK)

The graph displays the discriminating level of each variable for the selected class.

In [ ]:
class <- 2
variable <- "Function"

In [ ]:
plotDiscrimVar(resK, class = class, pkg = "plotly")

This chart summarizes the distribution of the selected variable for the selected class.

In [ ]:
plotDataBoxplot(resK, variable, class = class, grl = TRUE, pkg = "plotly")

### Probabilities

This chart shows the probability of assignment to a class for each individual. Individuals have been reordered in decreasing assignment probability. 

In [ ]:
heatmapTikSorted(resK)

### Advanced

Visualize in a *Gaussian-like way*, and onto R2, results of Gaussian or non-Gaussian based clustering.

In [ ]:
library(ClusVis)

In [ ]:
logTik <- getTik(resK, log = TRUE)
prop <- getProportion(resK)
resVisu <- clusvis(logTik, prop)

#### Component Interpretation

In [ ]:
plotDensityClusVisu(resVisu, add.obs = TRUE)

#### Observation Scatter-plot 

In [ ]:
plotDensityClusVisu(resVisu, add.obs = FALSE)